In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, current_timestamp, expr, date_trunc, when
import os

from datetime import datetime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1743716166877_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**move csv from landing to extract zone**

In [2]:
os.system(
    "aws s3 mv s3://00-arkham-challenge/fact_tables/sensor_readings/sensor_readings_2024_q1.csv s3://00-arkham-challenge/raw-zone-jagc/fact_tables/"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Unrecognized token 'Completed': was expecting (JSON String, Number, Array, Object or token 'null', 'true' or 'false')
 at [Source: (String)"Completed 1.9 MiB/1.9 MiB (9.3 MiB/s) with 1 file(s) remaining"; line: 1, column: 10]



In [3]:
# Read raw CSV file from S3
path = 's3://00-arkham-challenge/raw-zone-jagc/fact_tables/sensor_readings_2024_q1.csv'
sensor_readings = spark.read.csv(path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [5]:
print(sensor_readings.printSchema(), sensor_readings.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- timestamp: timestamp (nullable = true)
 |-- sensor_id: integer (nullable = true)
 |-- value: double (nullable = true)

None 56940

**import dim tables**

In [13]:
path = 's3://00-arkham-challenge/transform-zone-jagc/sensors/'
sensors = spark.read.parquet(path, header=True, inferSchema=True)

path = 's3://00-arkham-challenge/transform-zone-jagc/equipments/'
equipments = spark.read.parquet(path, header=True, inferSchema=True)

path = 's3://00-arkham-challenge/transform-zone-jag/plants/'
plants = spark.read.parquet(path, header=True, inferSchema=True)

path = 's3://00-arkham-challenge/transform-zone-jagc/time/'
time = spark.read.parquet(path, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'plants' is not defined
Traceback (most recent call last):
NameError: name 'plants' is not defined



In [15]:
print(sensors.printSchema(), sensors.count())
sensors.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- sensor_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- equipment_id: integer (nullable = true)

None 438
+---------+------------+-------+------------+
|sensor_id|        name|   unit|equipment_id|
+---------+------------+-------+------------+
|  3000001|Thermocouple|Celsius|     2000001|
|  3000002|Thermocouple|Celsius|     2000002|
|  3000003|Thermocouple|Celsius|     2000003|
|  3000004|Thermocouple|Celsius|     2000004|
|  3000005|Thermocouple|Celsius|     2000005|
+---------+------------+-------+------------+
only showing top 5 rows

In [16]:
print(equipments.printSchema(), equipments.count())
equipments.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- sensor_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- equipment_id: integer (nullable = true)

None 438
+---------+------------+-------+------------+
|sensor_id|        name|   unit|equipment_id|
+---------+------------+-------+------------+
|  3000001|Thermocouple|Celsius|     2000001|
|  3000002|Thermocouple|Celsius|     2000002|
|  3000003|Thermocouple|Celsius|     2000003|
|  3000004|Thermocouple|Celsius|     2000004|
|  3000005|Thermocouple|Celsius|     2000005|
+---------+------------+-------+------------+
only showing top 5 rows

In [17]:
print(plants.printSchema(), plants.count())
plants.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- equipment_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- plant_id: integer (nullable = true)

None 161
+------------+-----------+-------+--------+
|equipment_id|       name|   type|plant_id|
+------------+-----------+-------+--------+
|     2000001|Gas turbine|Turbine|    1001|
|     2000002|Gas turbine|Turbine|    1001|
|     2000003|Gas turbine|Turbine|    1001|
|     2000004|Gas turbine|Turbine|    1002|
|     2000005|Gas turbine|Turbine|    1002|
+------------+-----------+-------+--------+
only showing top 5 rows

In [30]:
print(time.printSchema(), time.count())
time.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- date_value: date (nullable = true)
 |-- time_id: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: long (nullable = true)

None 96432
+----------+----------+----+-----+---+----+
|date_value|   time_id|year|month|day|hour|
+----------+----------+----+-----+---+----+
|2020-01-01|1577880000|2020|    1|  1|  12|
|2020-01-02|1577966400|2020|    1|  2|  12|
|2020-01-03|1578052800|2020|    1|  3|  12|
|2020-01-04|1578139200|2020|    1|  4|  12|
|2020-01-05|1578225600|2020|    1|  5|  12|
+----------+----------+----+-----+---+----+
only showing top 5 rows

**transformations**

In [35]:
# Join sensor_readings with sensors: df1
df1 = sensor_readings.join(
    sensors.select('sensor_id', 'equipment_id'), 
    on = 'sensor_id',
    how="inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
(sensor_readings.count(), 
df1.count(), 
df1.filter(df1.equipment_id.isNull()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(56940, 56940, 0)

In [39]:
df1.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------+------------+
|sensor_id|          timestamp|  value|equipment_id|
+---------+-------------------+-------+------------+
|  3000001|2024-01-01 00:00:00|1050.94|     2000001|
|  3000001|2024-01-01 12:00:00|1026.39|     2000001|
|  3000001|2024-01-02 00:00:00|1129.72|     2000001|
+---------+-------------------+-------+------------+
only showing top 3 rows

In [40]:
# join df1 with equipments: df2
df2 = df1.join(
    equipments.select('equipment_id', 'plant_id'),
    on = 'equipment_id',
    how="inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------------+-------+------------+
|sensor_id|          timestamp|  value|equipment_id|
+---------+-------------------+-------+------------+
|  3000001|2024-01-01 00:00:00|1050.94|     2000001|
|  3000001|2024-01-01 12:00:00|1026.39|     2000001|
|  3000001|2024-01-02 00:00:00|1129.72|     2000001|
+---------+-------------------+-------+------------+
only showing top 3 rows

In [42]:
(df1.count(), 
 df2.count(), 
 df2.filter(df2.plant_id.isNull()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(56940, 56940, 0)

In [43]:
# reading date as Epoch
df2 = df2.withColumn("time_id", unix_timestamp(date_trunc("hour", col("timestamp"))))
# processed at timestamp as Epoch
df2 = df2.withColumn("processed_at", unix_timestamp(date_trunc("hour", current_timestamp())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(56940, 56940, 0)

In [45]:
df2.select('sensor_id','time_id', 'value', 'equipment_id', 'plant_id', 'processed_at').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------+-------+------------+--------+------------+
|sensor_id|   time_id|  value|equipment_id|plant_id|processed_at|
+---------+----------+-------+------------+--------+------------+
|  3000001|1704067200|1050.94|     2000001|    1001|  1744030800|
|  3000001|1704110400|1026.39|     2000001|    1001|  1744030800|
|  3000001|1704153600|1129.72|     2000001|    1001|  1744030800|
|  3000001|1704196800|1258.36|     2000001|    1001|  1744030800|
|  3000001|1704240000|  985.0|     2000001|    1001|  1744030800|
+---------+----------+-------+------------+--------+------------+
only showing top 5 rows

**load**

In [49]:
today = datetime.today()
day = today.day
month = today.month
year = today.year
print(f"Day: {day}, Month: {month}, Year: {year}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Day: 7, Month: 4, Year: 2025

In [51]:
path = 's3://00-arkham-challenge/transform-zone-jagc/sensor_readings/year={}/month={}/day={}/'.format(
    year, month, day
)
print(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://00-arkham-challenge/transform-zone-jagc/sensor_readings/year=2025/month=4/day=7/

In [52]:
# load fact table
df2.select(
    'sensor_id','time_id', 'value', 'equipment_id', 'plant_id', 'processed_at'
).write.parquet(path, mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://00-arkham-challenge/transform-zone-jagc/sensor_readings/year=2025/month=4/day=7/

In [54]:
print('done')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

done